In [1]:
%load_ext autoreload

In [2]:
import sys
import os
import time
import numpy as np
from functools import partial
import itertools
import mcubes
import visualizations_utils as viz_utils
# import iris_utils #TODO remove
from iris_plant_visualizer import IrisPlantVisualizer
import ipywidgets as widgets
from IPython.display import display


In [5]:
#pydrake imports
from pydrake.common import FindResourceOrThrow
from pydrake.multibody.parsing import LoadModelDirectives, Parser, ProcessModelDirectives
from pydrake.multibody.plant import MultibodyPlant, AddMultibodyPlantSceneGraph
from pydrake.systems.framework import DiagramBuilder
from pydrake.all import InverseKinematics, RevoluteJoint, RationalForwardKinematics
from pydrake.geometry.optimization import IrisOptionsRationalSpace, IrisInRationalConfigurationSpace, HPolyhedron, Hyperellipsoid
from pydrake.geometry import Role, GeometrySet, CollisionFilterDeclaration
import pydrake.symbolic as sym
from pydrake.all import MathematicalProgram, RigidTransform, RollPitchYaw
import meshcat
import pydrake.multibody.rational_forward_kinematics as rational_forward_kinematics
from time import time

# Build Plant


In [6]:
q0 = [0.0, 0.0, 0.0]
q_high = [ 1.7,  2.,  1.7]
q_low = [-i for i in q_high]


In [7]:
builder = DiagramBuilder()
plant, scene_graph = AddMultibodyPlantSceneGraph(builder, time_step=0.001)
parser = Parser(plant)
oneDOF_iiwa_asset = FindResourceOrThrow("drake/sos_iris_certifier/assets/oneDOF_iiwa7_with_box_collision.sdf")
twoDOF_iiwa_asset = FindResourceOrThrow("drake/sos_iris_certifier/assets/twoDOF_iiwa7_with_box_collision.sdf")

RuntimeError: Sought 'drake/sos_iris_certifier/assets/oneDOF_iiwa7_with_box_collision.sdf' in runfiles directory '/home/amice/.cache/bazel/_bazel_amice/32fdbbecfa8a7ce8feece95e48c42006/execroot/drake/bazel-out/k8-opt/bin/sos_iris_certifier/pinball_iiwas_snopt_iris.runfiles' but the file does not exist at that location nor is it on the manifest; perhaps a 'data = []' dependency is missing.

In [8]:
box_asset = FindResourceOrThrow("drake/sos_iris_certifier/assets/box_small.urdf")

models = []
models.append(parser.AddModelFromFile(box_asset))
models.append(parser.AddModelFromFile(twoDOF_iiwa_asset))
models.append(parser.AddModelFromFile(oneDOF_iiwa_asset))



# locs = [[0.,0.,0.],[0.,.5,0.],[0.,-.5,0.]]
locs = [[0.,0.,0.],[0.,.55,0.],[0.,-.55,0.]]
plant.WeldFrames(plant.world_frame(), plant.GetFrameByName("base", models[0]), RigidTransform(locs[0]))
plant.WeldFrames(plant.world_frame(), plant.GetFrameByName("iiwa_twoDOF_link_0", models[1]), RigidTransform(RollPitchYaw([0,0, -np.pi/2]).ToRotationMatrix(), locs[1]))
plant.WeldFrames(plant.world_frame(), plant.GetFrameByName("iiwa_oneDOF_link_0", models[2]), RigidTransform(RollPitchYaw([0,0, -np.pi/2]).ToRotationMatrix(), locs[2]))


plant.Finalize()

idx = 0
for model in models:
    for joint_index in plant.GetJointIndices(model):
        joint = plant.get_mutable_joint(joint_index)
        if isinstance(joint, RevoluteJoint):
            joint.set_default_angle(q0[idx])
            joint.set_position_limits(lower_limits= np.array([q_low[idx]]), upper_limits= np.array([q_high[idx]]))
            idx += 1
        
            
#q_low = plant.GetPositionLowerLimits().tolist()
#q_high = plant.GetPositionUpperLimits().tolist()

Ratfk = RationalForwardKinematics(plant)



RuntimeError: Sought 'drake/sos_iris_certifier/assets/box_small.urdf' in runfiles directory '/home/amice/.cache/bazel/_bazel_amice/32fdbbecfa8a7ce8feece95e48c42006/execroot/drake/bazel-out/k8-opt/bin/sos_iris_certifier/pinball_iiwas_snopt_iris.runfiles' but the file does not exist at that location nor is it on the manifest; perhaps a 'data = []' dependency is missing.

In [7]:
do_viz = True
visualizer = IrisPlantVisualizer(plant, builder, scene_graph, viz_role=Role.kIllustration)
diagram = visualizer.diagram
visualizer.visualize_collision_constraint(N = 30)

You can open the visualizer by visiting the following URL:
http://127.0.0.1:7006/static/
You can open the visualizer by visiting the following URL:
http://127.0.0.1:7007/static/
Connecting to meshcat-server at zmq_url=tcp://127.0.0.1:6006...
You can open the visualizer by visiting the following URL:
http://127.0.0.1:7006/static/
Connected to meshcat-server.


In [27]:
sliders = []
sliders.append(widgets.FloatSlider(min=q_low[0], max=q_high[0], value=0, description='q0'))
sliders.append(widgets.FloatSlider(min=q_low[1], max=q_high[1], value=0, description='q1'))
sliders.append(widgets.FloatSlider(min=q_low[2], max=q_high[2], value=0, description='q2'))

q = q0.copy()
def handle_slider_change(change, idx):
    q[idx] = change['new']
    #print(q, end="\r")
    visualizer.showres(q)
    
idx = 0
for slider in sliders:
    slider.observe(partial(handle_slider_change, idx = idx), names='value')
    idx+=1

for slider in sliders:
    display(slider)

visualizer.jupyter_cell()

Widget Javascript not detected.  It may not be installed or enabled properly.


Widget Javascript not detected.  It may not be installed or enabled properly.


Widget Javascript not detected.  It may not be installed or enabled properly.


In [9]:
#filter fused joints self collisions 
digaram = visualizer.diagram
context = visualizer.diagram_context
plant_context = visualizer.plant_context
sg_context = scene_graph.GetMyContextFromRoot(context)
inspector = scene_graph.model_inspector()

pairs = scene_graph.get_query_output_port().Eval(sg_context).inspector().GetCollisionCandidates()
print(len(inspector.GetCollisionCandidates()), "->", len(pairs))

gids = [gid for gid in inspector.GetGeometryIds(GeometrySet(inspector.GetAllGeometryIds()), Role.kProximity)]
get_name_of_gid = lambda gid : inspector.GetName(gid)
gids.sort(key=get_name_of_gid)
iiwa_oneDOF_gids = [gid for gid in gids if "iiwa7_oneDOF::" in get_name_of_gid(gid)]
iiwa_twoDOF_gids = [gid for gid in gids if "iiwa7_twoDOF::" in get_name_of_gid(gid)]

oneDOF_fused_col_geom = iiwa_oneDOF_gids[2:]
iiwa_oneDOF_fused_set = GeometrySet(oneDOF_fused_col_geom)
twoDOF_fused_col_geom = iiwa_twoDOF_gids[4:]
iiwa_twoDOF_fused_set = GeometrySet(twoDOF_fused_col_geom)
# print([get_name_of_gid(gid) for gid in oneDOF_fused_col_geom])
# print([get_name_of_gid(gid) for gid in twoDOF_fused_col_geom])
scene_graph.collision_filter_manager()\
            .Apply(CollisionFilterDeclaration().ExcludeWithin(iiwa_oneDOF_fused_set))
scene_graph.collision_filter_manager()\
            .Apply(CollisionFilterDeclaration().ExcludeWithin(iiwa_twoDOF_fused_set))
pairs = scene_graph.get_query_output_port().Eval(sg_context).inspector().GetCollisionCandidates()
print(len(inspector.GetCollisionCandidates()), "->", len(pairs))

150 -> 150
137 -> 150


In [10]:
def check_coverage_ret_points(regions, N = 5000):
    collision_free_points_t = np.zeros((N,3))
    col_func_handle_q = partial(visualizer.eval_cons, c=visualizer.collision_constraint, tol=1e-1)
    col_func_handle_t = lambda t: col_func_handle_q(Ratfk.ComputeQValue(t, np.array([0,0,0])))
    #col_func_handle_t(Ratfk.ComputeTValue(t, np.array([0,0,0])))

    #sample N collision free points
    for idx in range(N):
        if idx%500 ==0:
            print(idx)
        rand = np.random.rand(3)
        t = limits_t[0] + rand*(limits_t[1]-limits_t[0])
        while col_func_handle_t(t):
            rand = np.random.rand(3)
            t = limits_t[0] + rand*(limits_t[1]-limits_t[0])
        collision_free_points_t[idx, :] = t

    #check containment
    num_contained_in_regions = 0
    not_covered_points_list = []
    for pt in collision_free_points_t:
        covered = False
        for r in regions:
            if r.PointInSet(pt):
                num_contained_in_regions +=1
                covered = True
                break
        if not covered:
            not_covered_points_list.append(pt)
    not_covered_points = np.array(not_covered_points_list)
    frac_covered = num_contained_in_regions/N
    return frac_covered, num_contained_in_regions, collision_free_points_t, not_covered_points

def check_coverage(regions, N = 5000):
    frac_covered, num_contained_in_regions, _, _ = check_coverage_ret_points(regions, N)
    return frac_covered, num_contained_in_regions



# Setup IRIS Options and Generate Regions

In [26]:
# seed_points_q = np.array([[0.0, 0, 0], # zero config
#                         [0.0, -1.3, -1.3],  # start
#                         [0.9, -1.5, -0.8]     # goal
#                          ])
# seed_points_q = np.array([#[0.0, 0, 0], # zero config
#                         [0.8, 1.3, -0.8],  # START: blue low green up
# #                         [0.1, 0.9, -1.2],     # GOAL: green low other up
# #                         [0.2, 1.6, -0.6],
# #                         [0.5, -1.9, -0.9], #broken point
# #                           [-0.6,-1.3,-0.3],
# #                           [-0.9,-1.7,-1.3], #broken point
#                         #   [1.30, 1.50, -0.1],
# #                           [ 0, -0.39479112, -1.66596253] #bottom trough
#                          ])#[:,(0,1,2)]   # passing

seed_points_q = np.array([
                        [1.0, 0, 0.2],
                        [1.5, 1.7, -0.3],
                        [0.1, 0.9, -1.2],
                        [0.9, 1.9, -1.0],
                        [1.3, -0.8, -0.8], 
                        [1.0, 1.5, -1.7+1e-3],#[1.1, 1.5, -1.7+1e-3],
                        [0.7, -1.9, -0.6],
                        [-0.7,-1.6,-1.3],
                        [.5, 1.7, -1.3],
                        [1.6,-0.2, -1.2],
                        [0,-1.7,-1.7+1e-3],
                        [-0.5,-1.6,-0.8],
                        [0.7,-0.7,-1.6],
                        [-0.2,-0.4,-0.8]
                         ])


seed_points = np.array([Ratfk.ComputeTValue(seed_points_q[idx], np.zeros((3,)))\
                        for idx in range(seed_points_q.shape[0])])
if do_viz:
    visualizer.plot_seedpoints(seed_points)



#compute limits in t-space
limits_t = []
for q in [q_low, q_high]:
    limits_t.append(Ratfk.ComputeTValue(np.array(q), np.zeros((3,)) ))
    
starting_poly = HPolyhedron.MakeBox(limits_t[0], limits_t[1])
context = diagram.CreateDefaultContext()
q_star = np.zeros(3)
t_low = Ratfk.ComputeTValue(q_low, q_star) 
t_high = Ratfk.ComputeTValue(q_high, q_star) 
def check_empty_polytope(C, d):
    prog = MathematicalProgram()
    t = prog.NewContinuousVariables(C.shape[1], "t")

    prog.AddLinearConstraint(C,-np.inf*np.ones(C.shape[0]), d, t)
    prog.AddBoundingBoxConstraint(limits_t[0], limits_t[1], t)
    result = MosekSolver().Solve(prog)
    return result.is_success(), result.get_solution_result()

In [25]:
# RUN THIS CELL IF YOU WANT TO DO SNOPT IRIS
do_cpp_snopt_iris = False
do_python_snopt_iris = True
if do_cpp_snopt_iris and do_python_snopt_iris:
    raise ValueError("Make up your mind about snopt implementation")

iris_regions = []

if do_cpp_snopt_iris:
    iris_options = IrisOptionsRationalSpace()
    iris_options.require_sample_point_is_contained = True
    iris_options.iteration_limit = 20
    iris_options.configuration_space_margin = 1e-5
    iris_options.max_faces_per_collision_pair = 60
    iris_options.termination_threshold = -1
    iris_options.q_star = np.zeros(3)
    iris_options.relative_termination_threshold = 0.05
    iris_options.enable_ibex = False
    #deprecated
    iris_options.certify_region_with_sos_during_generation = False
    iris_options.certify_region_with_sos_after_generation = False

    for i, s in enumerate(seed_points):
        q = Ratfk.ComputeTValue(s, np.zeros((3,)))
        plant.SetPositions(plant.GetMyMutableContextFromRoot(context), q)
        if False:
            #starting_hpolyhedron = regions[i-1]
            r = IrisInRationalConfigurationSpace (plant, plant.GetMyContextFromRoot(context),
                                                  iris_options, starting_hpolyhedron)
        else:
            r = IrisInRationalConfigurationSpace(plant, plant.GetMyContextFromRoot(context), iris_options)
        iris_regions.append(r)
        print(f'Completed region: {i+1}/{len(seed_points)}')
        print(f"Sample point contained = {np.all(r.A()@s <= r.b())}")
        print(f"Sample point contained componentwise = {r.A()@s <= r.b()}")
        print()
        
if do_python_snopt_iris:
    iris_kwargs = {
    'iris_starting_ellipse_vol': 1e-4,
    'iris_plane_pullback': 1e-5,
    'iris_max_faces': -1
    }
    iris_generator = CertifiedIrisRegionGenerator(visualizer.diagram, plant, scene_graph, **iris_kwargs)
    iris_regions, ellipses = iris_generator.iris_in_rational_space(seed_points, termination_threshold = 2e-4)

    
if do_viz:
    visualizer.plot_regions(iris_regions, ellipses=None, region_suffix='_iris_default')

check_frac_contained = True   
if check_frac_contained:
    frac_covered, num_contained_in_regions = check_coverage(iris_regions, N = int(5e3))
    print(f"frac covered = {frac_covered}")

snopt_example=[ 5.65443403e-01  5.12827994e-04 -2.26428521e-01], growth = 10714082.190841291
snopt_example=[-0.51857234  0.          0.10033467], growth = 113395840.51069826
snopt_example=[ 0.97859405 -0.35065326  0.10033467], growth = 30983369.695521962
snopt_example=[0.54630249 0.         0.32750925], growth = 5160829.045531731
snopt_example=[0.54630249 0.         0.19793377], growth = 952558.3631171323
snopt_example=[-0.06445007  0.26762072  0.10033467], growth = 44463953.57739997
snopt_example=[ 0.9633577  -0.35296364  0.10033467], growth = 29851837.980504584
snopt_example=[-0.48690827 -0.69652023  0.10033467], growth = 155266490.9352093
snopt_example=[-0.05368608  0.27610322  0.10033467], growth = 43621926.80714406
0
1
Time:   1.79 	Volume:   0.67 	Center: [ 0.44990841  0.13491412 -0.0175258 ]
snopt_example=[ 1.13832271  0.72465315 -0.36914321], growth = 26139401.272062868
snopt_example=[ 1.13832271  0.79330675 -0.46686547], growth = 26146425.525782645
snopt_example=[ 1.13832271  

In [13]:
plot_not_covered_samples = False
if plot_not_covered_samples and do_viz:
    frac_covered, num_contained_in_regions, collision_free_points_t, not_covered_points = check_coverage_ret_points(iris_regions, N = int(10e3))
    print(f"frac covered = {frac_covered}")
    print(f"num not covered = {not_covered_points.shape}")
    plotted = 0
    for i, t in enumerate(not_covered_points):
        if np.random.rand() < 1.0:
            visualizer.vis2['iris']['collision_free_points'][str(plotted)].set_object(
                                meshcat.geometry.Sphere(0.01), meshcat.geometry.MeshLambertMaterial(color=0xE3EB0C))
            visualizer.vis2['iris']['collision_free_points'][str(plotted)].set_transform(
                            meshcat.transformations.translation_matrix(t))
            plotted += 1
    


In [14]:
do_large_sample_cover_test = False
if do_large_sample_cover_test:
    frac_covered, num_contained_in_regions = check_coverage(iris_regions, N = int(5e4))
    print(f"frac covered = {frac_covered}")


# Certify Regions

In [15]:
cspace_free_region = rational_forward_kinematics.CspaceFreeRegion(diagram, plant, scene_graph,
                                   rational_forward_kinematics.SeparatingPlaneOrder.kAffine,
                                   rational_forward_kinematics.CspaceRegionType.kGenericPolytope)
filtered_collision_pairs = set()
solver_options = mp.SolverOptions()
# solver_options.SetOption(mp.CommonSolverOption.kPrintToConsole, 1)

binary_search_options = rational_forward_kinematics.BinarySearchOption()
binary_search_options.epsilon_max = 0.0
binary_search_options.epsilon_min = -.12
binary_search_options.max_iters = 15
binary_search_options.search_d = True
bilinear_alternation_options = rational_forward_kinematics.BilinearAlternationOption()
bilinear_alternation_options.max_iters = 20
bilinear_alternation_options.lagrangian_backoff_scale = 1e-6
bilinear_alternation_options.polytope_backoff_scale = 1e-6
def certify_region(region, seed_point, verbose = False):
    C = region.A()
    d = region.b()
    binary_search_options.epsilon_min = FindEpsilonLower(C, d, t_low, t_high, seed_point)
    t0 = time()
    cspace_free_region_solution_binary_search = cspace_free_region.CspacePolytopeBinarySearch(q_star,
                                                                     filtered_collision_pairs,
                                                                     C, d, binary_search_options, 
                                                                     solver_options, seed_point)
    t1 = time()
    if verbose:
        print(f"Time to certify binary = {t1-t0}")
    C_binary, d_binary = cspace_free_region_solution_binary_search.C, cspace_free_region_solution_binary_search.d
    cspace_free_region_solution_bilinear = cspace_free_region.CspacePolytopeBilinearAlternation(q_star,
                                                                     filtered_collision_pairs,
                                                                     C_binary,
                                                                     d_binary-1e-4,
                                                                     bilinear_alternation_options,
                                                                     solver_options, seed_point)
    t2 = time()
    if verbose:
        print(f"Time to certify alt = {t2-t1}")
    C_alt, d_alt = cspace_free_region_solution_bilinear.C, cspace_free_region_solution_bilinear.d
    region_binary = HPolyhedron(C_binary, d_binary)
    region_alt = HPolyhedron(C_alt, d_alt)
    return region_binary, region_alt


In [19]:
regions_binary, regions_alt = [], []
start = 11
end = -1
regions = iris_regions[start:end]
seed_points_to_cert = seed_points[start:end]
for i, (r,s) in enumerate(zip(regions, seed_points_to_cert)):
    print(f"Starting region {i+1}/{len(regions)}")
    r_binary, r_alt = certify_region(r,s)
    regions_binary.append(r_binary)
    regions_alt.append(r_alt)
    if do_viz:
        visualizer.plot_regions(regions_binary, ellipses=None, region_suffix='_binary_cert')
        visualizer.plot_regions(regions_alt, ellipses=None, region_suffix='_alt_cert')
    print()

# problem regions: 5,8 ...

[2022-04-13 16:14:35.528] [console] [warning] Find Lagrangian failed
[2022-04-13 16:14:39.175] [console] [info] Lagrangian SOS takes 1.3177909851074219 seconds


Starting region 1/2


[2022-04-13 16:14:39.285] [console] [info] max(power(det(P), 1/4))=0.4504319545504092, solver_time 0.007275104522705078
[2022-04-13 16:14:41.259] [console] [info] search d is successful = true
[2022-04-13 16:14:41.279] [console] [info] max(power(det(P), 1/4))=0.4624011938822777, solver_time 0.00864720344543457
[2022-04-13 16:14:45.185] [console] [info] Lagrangian SOS takes 1.5421531200408936 seconds
[2022-04-13 16:14:45.324] [console] [info] max(power(det(P), 1/4))=0.4781694151167616, solver_time 0.0202939510345459
[2022-04-13 16:14:47.412] [console] [info] search d is successful = true
[2022-04-13 16:14:47.438] [console] [info] max(power(det(P), 1/4))=0.48497388197787694, solver_time 0.01304483413696289
[2022-04-13 16:14:47.466] [console] [info] epsilon=-0.02620378994520936 is feasible
[2022-04-13 16:14:47.467] [console] [info] reset eps_min=0.0, eps_max=0.026203720209618142
[2022-04-13 16:14:50.888] [console] [info] Lagrangian SOS takes 1.2889580726623535 seconds
[2022-04-13 16:14:51


Starting region 2/2


[2022-04-13 16:16:28.502] [console] [info] max(power(det(P), 1/4))=0.3300580767138361, solver_time 0.013998985290527344
[2022-04-13 16:16:30.641] [console] [info] search d is successful = true
[2022-04-13 16:16:30.675] [console] [info] max(power(det(P), 1/4))=0.33342104632906383, solver_time 0.022089004516601562
[2022-04-13 16:16:35.053] [console] [info] Lagrangian SOS takes 1.6735498905181885 seconds
[2022-04-13 16:16:35.193] [console] [info] max(power(det(P), 1/4))=0.3710427945417672, solver_time 0.015472888946533203
[2022-04-13 16:16:37.015] [console] [info] search d is successful = false
[2022-04-13 16:16:37.030] [console] [info] epsilon=-0.03219285843080713 is feasible
[2022-04-13 16:16:37.031] [console] [info] reset eps_min=0.0, eps_max=0.03219285843080716
[2022-04-13 16:16:42.430] [console] [info] Lagrangian SOS takes 2.3625059127807617 seconds
[2022-04-13 16:16:42.568] [console] [info] max(power(det(P), 1/4))=0.3909718630593891, solver_time 0.02159905433654785
[2022-04-13 16:16

In [ ]:
# regions_binary, regions_alt = [], []
# for r in iris_regions:
#     r_binary, r_alt = certify_region(r)
#     regions_binary.append(r_binary)
#     regions_alt.append(r_alt)
# if do_viz:
#     visualizer.plot_regions(regions_binary, ellipses=None, region_suffix='_binary_cert')
#     visualizer.plot_regions(regions_alt, ellipses=None, region_suffix='_alt_cert')

# DO NOT RUN UNDER HERE

In [ ]:
seed_point_list = [s for s in seed_points] 
print(len(seed_point_list))
C_mats = [r.A() for r in regions_default]
d_vects = [r.b() for r in regions_default]
t0 = time()
cspace_free_region_solution_interleaved_vect = cspace_free_region.InterleavedCSpacePolytopeSearchForSeedPoints(
                                                                    q_star,
                                                                     filtered_collision_pairs,
                                                                     C_mats, d_vects, num_round_robin_rounds, 
                                                                     interleaved_options_list,
                                                                     solver_options, seed_point_list, 
                                        context = None)#plant.GetMyMutableContextFromRoot(context))
# cspace_free_region_solution_round_robin_vect = cspace_free_region.CspacePolytopeRoundRobinBisectionSearchForSeedPoints(
#                                                                     q_star,
#                                                                      filtered_collision_pairs,
#                                                                      C_mats, d_vects, num_round_robin_rounds, 
#                                                                      vector_bisection_options_list,
#                                                                      solver_options, seed_point_list)
t1 = time()


In [26]:
print(seed_points)

[[ 0.25534192 -1.39838259 -0.48305507]
 [-0.48305507 -1.13833271 -0.7602044 ]]


In [27]:
print(f"Completed points in {t1-t0}s")
regions_round_robin = []
ellipses_round_robin = []
for sol in cspace_free_region_solution_interleaved_vect:
    r = HPolyhedron(np.vstack([starting_poly.A(),sol.C]),
                np.hstack([starting_poly.b(), sol.d])
                )
    e = Hyperellipsoid(np.linalg.inv(sol.P), sol.q)

    regions_round_robin.append(r)
    ellipses_round_robin.append(e)
if do_viz:
    visualizer.plot_regions(regions_round_robin, 
                            ellipses=None, region_suffix='_interleaved2')

Completed points in 2610.1760251522064s


In [28]:
import visualizations_utils as vis_utils
import cdd
from scipy.spatial import Delaunay
#def plot_regions2(regions):
def plot_3d_poly2(region, vis, name, mat = None, verbose = False):
    # First, prune region
    #region = region.ReduceInequalities()

    #region = region.ReduceInequalities()

    def project_and_triangulate(pts):
        n = np.cross(pts[0,:]-pts[1,:],pts[0,:]-pts[2,:])
        n = n / np.linalg.norm(n)
        if not np.abs(n[2]) < 1e-3:  # normal vector not in the XY-plane
            pts_prj = pts[:,:2]
        else:
            if np.abs(n[0]) < 1e-3:  # normal vector not in the Y-direction
                pts_prj = pts[:,[0,2]]
            else:
                pts_prj = pts[:,1:3]
        tri = Delaunay(pts_prj)
        return tri.simplices

    # Find feasible point in region
    # prog = MathematicalProgram()
    # x = prog.NewContinuousVariables(3)
    # prog.AddConstraint(le(region.A()@x, region.b()))
    # result = Solve(prog)
    # if result.is_success():
    #    x0 = result.GetSolution()
    # else:
    #    print("Solve failed. No feasible point found in region.")
    x0 = region.MaximumVolumeInscribedEllipsoid().center()
    A = region.A()
    b = region.b() - A@x0

    # Ax <= b must be in form [b -A]
    b_minusA = np.concatenate((b.reshape(-1,1),-A),axis=1)

    matrix = cdd.Matrix(b_minusA)
    matrix.rep_type = cdd.RepType.INEQUALITY
    poly = cdd.Polyhedron(matrix)
    # Get vrep
    gen = poly.get_generators()

    # [t vertices]
    t_v = np.array(gen)
    vertices = t_v[:,1:] + x0  # vertices need to be moved back

    # only keep nonempty facets
    facets_with_duplicates = [np.array(list(facet)) for facet in poly.get_input_incidence() if len(list(facet))>2]

    # if facet is subset of any other facet, remove
    remove_idxs = []
    for idx, facet in enumerate(facets_with_duplicates):
        for other_idx, other_facet in enumerate(facets_with_duplicates):
            if (not (idx == other_idx)) and set(facet).issubset(set(other_facet)):
                remove_idxs.append(idx)
    if remove_idxs:  # if list not empty 
        facets = list(np.delete(np.array(facets_with_duplicates), np.array(remove_idxs)))
    else:
        facets = facets_with_duplicates

    mesh_vertices = []
    mesh_triangles = []
    
    count = 0
    for idx, facet in enumerate(facets):
        tri = project_and_triangulate(vertices[facet])
        verts = vertices[facet]
        idxs = tri+count
    #    print('it:', idx)
    #   print(len(verts))
    #    print(len(idxs))
        mesh_vertices.append(verts)
        mesh_triangles.append(idxs)

        count += vertices[facet].shape[0]

    # print('end')
    mesh_vertices = np.concatenate(mesh_vertices, 0)
    mesh_triangles = np.concatenate(mesh_triangles, 0)

    if mat is None:
        mat = meshcat.geometry.MeshLambertMaterial(color=0x000000 , wireframe=False)
        mat.opacity = 0.3
    
    vis[name].set_object(
                meshcat.geometry.TriangularMeshGeometry(mesh_vertices, mesh_triangles),
                mat)

In [29]:
def plot_regions2(vis, regions, ellipses = None, opacity = 0.8, region_suffix=''):
    colors = vis_utils.n_colors(len(regions))
    for i, region in enumerate(regions):
        print(i)
        c = colors[i]
        mat = meshcat.geometry.MeshLambertMaterial(color=vis_utils.rgb_to_hex(c), wireframe=False)
        mat.opacity = opacity
        plot_3d_poly2(region=region,
                           vis=vis['iris']['regions'+region_suffix],
                           name=str(i),
                           mat=mat)
        #plot_3d_poly_marchingcubes(region=region,
        #                    resolution=30,
         #                   vis=vis['iris']['regions'+region_suffix],
         #                   name=str(i),
          #                  mat=mat)
        if ellipses is not None:
            C = ellipses[i].A()  # [:, (0,2,1)]
            d = ellipses[i].center()  # [[0,2,1]]
            radii, R = np.linalg.eig(C.T @ C)
            R[:, 0] = R[:, 0] * np.linalg.det(R)
            Rot = RotationMatrix(R)

            transf = RigidTransform(Rot, d)
            mat = meshcat.geometry.MeshLambertMaterial(color=vis_utils.rgb_to_hex(c), wireframe=True)
            mat.opacity = 0.15
            vis['iris']['ellipses'+region_suffix][str(i)].set_object(
                meshcat.geometry.Ellipsoid(np.divide(1, np.sqrt(radii))),
                mat)

            vis['iris']['ellipses'+region_suffix][str(i)].set_transform(transf.GetAsMatrix4())

In [30]:
print(f"Completed points in {t1-t0}s")
regions_round_robin = []
ellipses_round_robin = []
for sol in cspace_free_region_solution_interleaved_vect:
    r = HPolyhedron(np.vstack([starting_poly.A(),sol.C]),
                np.hstack([starting_poly.b(), sol.d])
                )
    e = Hyperellipsoid(np.linalg.inv(sol.P), sol.q)

    regions_round_robin.append(r)
    ellipses_round_robin.append(e)
if do_viz:
    plot_regions2(visualizer.vis2, regions_round_robin, 
                            ellipses=None, region_suffix='_interleaved3')

Completed points in 2610.1760251522064s
0
1


In [77]:
visualizer.plot_seedpoints(seed_points_TT)

In [76]:
seed_points_TT = np.array([Ratfk.ComputeTValue(seed_points_[idx], np.zeros((3,)))\
                        for idx in range(seed_points_q.shape[0])])

In [54]:
for r in regions_round_robin:
    print(r.MaximumVolumeInscribedEllipsoid().center())

[ 0.01687129 -0.01288147 -0.02802416]
[ 0.26241809  0.98004433 -0.12550275]
[ 4.88210523e-04  4.30476013e-02 -5.06340370e-01]
[ 0.07314422  0.75060654 -0.22550694]
[-0.04887298 -1.19159415 -0.09069353]
[-0.2409991  -0.98683032 -0.12701035]
[-0.35778394 -0.72233889 -0.55402954]
[ 0.6943619   1.04165496 -0.05417009]


In [26]:
print(f"Completed points in {t1-t0}s")
regions_round_robin = []
ellipses_round_robin = []
for i, sol in enumerate(cspace_free_region_solution_interleaved_vect):
    dtmp = sol.d+sol.C@seed_points[i,:]
    
    r = HPolyhedron(np.vstack([starting_poly.A(),sol.C*2]),
                np.hstack([starting_poly.b(), dtmp])
                )
    e = Hyperellipsoid(np.linalg.inv(sol.P), sol.q)

    regions_round_robin.append(r)
    ellipses_round_robin.append(e)
if do_viz:
    visualizer.plot_regions(regions_round_robin, 
                            ellipses=None, region_suffix='_interleaved_small')

Completed points in 5785.018831014633s


In [27]:
from datetime import datetime

# datetime object containing current date and time
now = datetime.now()
 
print("now =", now)

# dd/mm/YY H:M:S
dt_string = now.strftime("%d/%m/%Y %H:%M:%S")
print("finished running at =", dt_string)	

now = 2022-02-13 17:35:10.574604
finished running at = 13/02/2022 17:35:10


In [28]:
visualizer.vis2["regions_interleaved"]

<Visualizer using: <meshcat.visualizer.ViewerWindow object at 0x7fe2748d6eb0> at path: <meshcat.path.Path object at 0x7fe257e406d0>>

In [30]:
N = 50000
collision_free_points_t = np.zeros((N,3))
col_func_handle_t = partial(visualizer.eval_cons, c=visualizer.collision_constraint, tol=0.01)
#col_func_handle_t(Ratfk.ComputeTValue(t, np.array([0,0,0])))

#sample N collision free points
for idx in range(N):
        if idx%400 ==0:
            print(idx)
        rand = np.random.rand(3)
        t = limits_t[0] + rand*(limits_t[1]-limits_t[0])
        while col_func_handle_t(t):
            rand = np.random.rand(3)
            t = limits_t[0] + rand*(limits_t[1]-limits_t[0])
        collision_free_points_t[idx, :] = t

#check containment
num_contained_in_regions = 0
for pt in collision_free_points_t:
    for region in regions_round_robin:
        if region.PointInSet(pt):
            num_contained_in_regions +=1
            break
        
print("Num col free points: ", N, "Num contained in round robin regions: ", num_contained_in_regions)

0
400
800
1200
1600
2000
2400
2800
3200
3600
4000
4400
4800
5200
5600
6000
6400
6800
7200
7600
8000
8400
8800
9200
9600
10000
10400
10800
11200
11600
12000
12400
12800
13200
13600
14000
14400
14800
15200
15600
16000
16400
16800
17200
17600
18000
18400
18800
19200
19600
20000
20400
20800
21200
21600
22000
22400
22800
23200
23600
24000
24400
24800
25200
25600
26000
26400
26800
27200
27600
28000
28400
28800
29200
29600
30000
30400
30800
31200
31600
32000
32400
32800
33200
33600
34000
34400
34800
35200
35600
36000
36400
36800
37200
37600
38000
38400
38800
39200
39600
40000
40400
40800
41200
41600
42000
42400
42800
43200
43600
44000
44400
44800
45200
45600
46000
46400
46800
47200
47600
48000
48400
48800
49200
49600
Num col free points:  50000 Num contained in round robin regions:  872


0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0


In [33]:
visualizer.q_star = 0.0*np.zeros((3,1))
print(visualizer.q_star.dtype)
print(t.dtype)
visualizer.col_func_handle_rational(t.reshape(-1,1))

float64
float64


TypeError: ComputeQValue(): incompatible function arguments. The following argument types are supported:
    1. (self: pydrake.multibody.rational_forward_kinematics.RationalForwardKinematics, t_val: numpy.ndarray[numpy.float64[m, 1]], q_star_val: numpy.ndarray[numpy.float64[m, 1]]) -> numpy.ndarray[numpy.float64[m, 1]]

Invoked with: <pydrake.multibody.rational_forward_kinematics.RationalForwardKinematics object at 0x7f3a726ae230>, array([[[ 0.43109738],
        [-0.84842998],
        [ 0.96040215]]]), array([[0.],
       [0.],
       [0.]])

In [11]:
rand = np.random.rand(3)
pos_samp = limits_t[0] + rand*(limits_t[1]-limits_t[0])


1.0

# Don't run after here

In [36]:
cspace_free_region_solution_round_robin = cspace_free_region.CspacePolytopeRoundRobinBisectionSearch(q_star,
                                                                     filtered_collision_pairs,
                                                                     C, d, 1, 
                                                                     vector_bisection_search_options,
                                                                     solver_options, s)

[2022-02-09 11:46:31.589] [console] [info] Found Lagrangian multiplier and separating planes
[2022-02-09 11:46:34.425] [console] [info] bilinear alt on d succeeded
[2022-02-09 11:46:34.475] [console] [info] Ineq 1/12 in round 1
[2022-02-09 11:46:34.476] [console] [info] epsilon exceeds components of epsilon max in some components, increasing epsilon max
[2022-02-09 11:46:34.491] [console] [info] output of eps_redundant_max is -3.436515294552578e-09
[2022-02-09 11:46:34.492] [console] [warning] round robin bisection search failed: epsilon max is less than epsilon min.
 difference is = 
     -0.25
-0.238571
-0.225191
 -0.20519
    -0.25
    -0.25
-0.239698
    -0.25
    -0.25
    -0.25
    -0.25
    -0.25
[2022-02-09 11:46:34.493] [console] [info] Ineq 2/12 in round 1
[2022-02-09 11:46:34.511] [console] [info] output of eps_redundant_max is 6.756142090225126e-09
[2022-02-09 11:46:34.512] [console] [info] Ineq 3/12 in round 1
[2022-02-09 11:46:34.529] [console] [info] output of eps_redund

[2022-02-09 11:47:44.971] [console] [info] Found Lagrangian multiplier and separating planes
[2022-02-09 11:47:47.809] [console] [info] bilinear alt on d succeeded
[2022-02-09 11:47:47.841] [console] [info] epsilon=-0.1569430869108956 is feasible
[2022-02-09 11:47:47.842] [console] [info] epsilon exceeds components of epsilon max in some components, increasing epsilon max
[2022-02-09 11:47:47.851] [console] [info] output of eps_redundant_max is 0.0007903398544878426
[2022-02-09 11:47:47.852] [console] [info] reset eps_min=-0.15694308925800982, eps_max=-0.15615274940352197
[2022-02-09 11:47:47.853] [console] [info] current gap is epsilon_gap=0.0007903398544878426
[2022-02-09 11:47:53.058] [console] [info] Found Lagrangian multiplier and separating planes
[2022-02-09 11:47:55.860] [console] [info] bilinear alt on d succeeded
[2022-02-09 11:47:55.892] [console] [info] epsilon=-0.1565479193307659 is feasible
[2022-02-09 11:47:55.893] [console] [info] epsilon exceeds components of epsilon m

In [29]:
cspace_free_region_solution_bilinear = cspace_free_region.CspacePolytopeBilinearAlternation(q_star,
                                                                     filtered_collision_pairs,
                                                                     cspace_free_region_solution_round_robin.C,
                                                                    0.75*cspace_free_region_solution_round_robin.d,
                                                                     bilinear_alternation_options,
                                                                     solver_options, s)

[2022-02-08 19:34:36.721] [console] [info] Found Lagrangian multiplier and separating planes
[2022-02-08 19:34:36.723] [console] [info] Lagrangian step time 3.045 s
[2022-02-08 19:34:36.751] [console] [info] max(power(det(P), 1/4))=0.2071381570004865, solver_time 0.018611907958984375
[2022-02-08 19:34:36.753] [console] [info] cost improvement inf
[2022-02-08 19:34:38.615] [console] [info] Iter: 0, polytope step cost -0.008828768121858371, solver time 0.48531293869018555
[2022-02-08 19:34:41.859] [console] [info] Found Lagrangian multiplier and separating planes
[2022-02-08 19:34:41.860] [console] [info] Lagrangian step time 3.218 s
[2022-02-08 19:34:41.881] [console] [info] max(power(det(P), 1/4))=0.2105033555416147, solver_time 0.013540029525756836
[2022-02-08 19:34:41.882] [console] [info] cost improvement 0.0033651985411282115
[2022-02-08 19:34:43.780] [console] [info] Iter: 1, polytope step cost -0.008522369595631428, solver time 0.532606840133667
[2022-02-08 19:34:46.920] [console

[2022-02-08 19:36:00.640] [console] [info] Iter: 16, polytope step cost -0.00557567747292513, solver time 0.38878607749938965
[2022-02-08 19:36:04.137] [console] [info] Found Lagrangian multiplier and separating planes
[2022-02-08 19:36:04.141] [console] [info] Lagrangian step time 3.472 s
[2022-02-08 19:36:04.172] [console] [info] max(power(det(P), 1/4))=0.24350116763464993, solver_time 0.01593494415283203
[2022-02-08 19:36:04.174] [console] [info] cost improvement 0.001400599503426303
[2022-02-08 19:36:05.966] [console] [info] Iter: 17, polytope step cost -0.005466929346269455, solver time 0.3860509395599365
[2022-02-08 19:36:09.113] [console] [info] Found Lagrangian multiplier and separating planes
[2022-02-08 19:36:09.114] [console] [info] Lagrangian step time 3.119 s
[2022-02-08 19:36:09.142] [console] [info] max(power(det(P), 1/4))=0.24485582644352566, solver_time 0.00913095474243164
[2022-02-08 19:36:09.145] [console] [info] cost improvement 0.001354658808875725
[2022-02-08 19:3

In [31]:
if do_viz:
    visualizer.plot_regions([HPolyhedron(cspace_free_region_solution_bilinear.C,
                                         cspace_free_region_solution_bilinear.d)], 
                            ellipses=None, region_suffix='_bilinear')

In [15]:
regions_round_robin = []
ellipses_round_robin = []
for sol in cspace_free_region_solution_round_robin_vect:
    r = HPolyhedron(np.vstack([starting_poly.A(),sol.C]),
                np.hstack([starting_poly.b(), sol.d])
                )
    e = Hyperellipsoid(np.linalg.inv(sol.P), sol.q)

    regions_round_robin.append(r)
    ellipses_round_robin.append(e)
if do_viz:
    visualizer.plot_regions(regions_round_robin, 
                            ellipses=ellipses_round_robin, region_suffix='_round_robin')

In [22]:
if do_viz:
    visualizer.plot_regions([HPolyhedron(cspace_free_region_solution.C, 0.5*cspace_free_region_solution.d)], 
                            ellipses=None, region_suffix='_round_robin')

In [26]:
cspace_free_region_solution_vector_bisection = cspace_free_region.CspacePolytopeBisectionSearchVector(q_star,
                                                                     filtered_collision_pairs,
                                                                     C, d, 
                                                                     vector_bisection_search_options,
                                                                     solver_options, s)

[2022-02-07 15:27:37.125] [console] [info] Found Lagrangian multiplier and separating planes
[2022-02-07 15:27:37.194] [console] [info] max(power(det(P), 1/4))=-nan, solver_time 0.05913114547729492
[2022-02-07 15:27:40.385] [console] [info] bilinear alt on d succeeded
[2022-02-07 15:27:40.411] [console] [info] max(power(det(P), 1/4))=1.195363453968912e-06, solver_time 0.014796972274780273
[2022-02-07 15:27:45.796] [console] [warning] Cannot find Lagrangian multiplier and separating planes for 
(iiwa7_twoDOF::iiwa_twoDOF_link_3_collision, iiwa7_oneDOF::iiwa_oneDOF_link_7_collision)
(iiwa7_twoDOF::iiwa_twoDOF_link_4_collision, iiwa7_oneDOF::iiwa_oneDOF_link_6_collision)
(iiwa7_twoDOF::iiwa_twoDOF_link_4_collision, iiwa7_oneDOF::iiwa_oneDOF_link_7_collision)

[2022-02-07 15:27:45.797] [console] [info] total iter=1, feasible iter = 0
[2022-02-07 15:27:51.001] [console] [warning] Cannot find Lagrangian multiplier and separating planes for 
(iiwa7_twoDOF::iiwa_twoDOF_link_4_collision, iiwa7_

In [29]:
if do_viz:
    visualizer.plot_regions([HPolyhedron(cspace_free_region_solution_vector_bisection.C, 
                                         cspace_free_region_solution_vector_bisection.d)], 
                            ellipses=None, region_suffix='_vector_bisection')

(True, <SolutionResult.kSolutionFound: 0>)


In [96]:
i

2